In [98]:
#!pip install geopandas
#!pip install openpyxl
#!pip install folium

In [81]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
from openpyexcel import load_workbook
import numpy as np

import folium 
from folium import Circle,Choropleth,Marker
from folium.plugins import HeatMap,MarkerCluster

In [82]:
full_workbook=load_workbook('CHANDRAPUR_DATA_new.xlsx')
print(full_workbook.sheetnames)

['BASE DATA', 'Sheet1', 'Rural DC', 'ghughus']


In [83]:
chandrapur_div_df=pd.read_excel('CHANDRAPUR_DATA_new.xlsx',sheet_name='BASE DATA')
#chanda_district_df=pd.read_excel('df.xlsx',sheet_name='CHANDRAPUR CIRCLE')
chandrapur_div_df['SECTION_NAME'].value_counts()
chandrapur_div_df=pd.read_excel('CHANDRAPUR_DATA_new.xlsx',sheet_name='Rural DC')

In [84]:
chandrapur_div_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SECTION_NAME   50 non-null     object 
 1   CONSUMER NO    50 non-null     int64  
 2   CONSUMER NAME  50 non-null     object 
 3   ADDRESS        50 non-null     object 
 4   SANC LOAD      50 non-null     float64
 5   METER NO       50 non-null     int64  
 6   MOBILE         50 non-null     int64  
 7   DTC            50 non-null     object 
 8   LATITUDE       50 non-null     float64
 9   LONGITUDE      50 non-null     float64
 10  Googlemap      47 non-null     object 
dtypes: float64(3), int64(3), object(5)
memory usage: 4.4+ KB


In [85]:
cate_col=chandrapur_div_df.select_dtypes(exclude=np.number)
cate_col.columns

Index(['SECTION_NAME', 'CONSUMER NAME', 'ADDRESS', 'DTC', 'Googlemap'], dtype='object')

In [93]:
chandrapur_div_df_sorted=chandrapur_div_df[['SECTION_NAME','CONSUMER NAME','ADDRESS','MOBILE','METER NO','LATITUDE','LONGITUDE','Googlemap']]
#chanda_district_df_sorted=chanda_district_df[['DIVISION','SUB_DIVISION','SECTION_NAME','CONSUMER NAME','ADDRESS','MOBILE','METER NO','LATITUDE','LONGITUDE']]

In [94]:
for val in chandrapur_div_df_sorted['SECTION_NAME'].value_counts().index:
    print(val)

Rural DC
MIDC DC


In [95]:
df_test=chandrapur_div_df.groupby('SECTION_NAME')[['LATITUDE','LONGITUDE']].value_counts()
df_test

SECTION_NAME  LATITUDE   LONGITUDE
MIDC DC       19.962301  79.256849    1
              19.963583  79.261096    1
              19.964466  79.259874    1
Rural DC      20.109833  79.349333    1
              20.053552  79.455215    1
              20.065682  79.295484    1
              20.067614  79.285786    1
              20.067664  79.285773    1
              20.068071  79.293217    1
              20.082982  79.281332    1
              20.088772  79.274621    1
              20.095502  79.270347    1
              20.100680  79.269782    1
              20.100725  79.272775    1
              20.212630  79.254455    1
              20.216480  79.202119    1
              20.233120  79.271569    1
              20.246140  79.276480    1
              20.250910  79.273382    1
              20.311601  79.208220    1
              20.318670  79.175496    1
              20.350680  79.222935    1
              20.356130  79.224290    1
              20.435430  79.171566    1
     

In [96]:
df_ballarshah=chandrapur_div_df_sorted[chandrapur_div_df_sorted['DIVISION']=='BALLARSHAH DIVISION']
df_ballarshah
df_chandrapur=chandrapur_div_df_sorted[chandrapur_div_df_sorted['DIVISION']=='CHANDRAPUR DIVSION']
df_chandrapur

KeyError: 'DIVISION'

In [97]:
#converting the df into gdf
geometry=[Point(xy)for xy in zip(chandrapur_div_df_sorted['LONGITUDE'],chandrapur_div_df_sorted['LATITUDE'])]
geometry2=[Point(xy)for xy in zip(df_ballarshah['LONGITUDE'],df_ballarshah['LATITUDE'])]
geometry3=[Point(xy)for xy in zip(df_chandrapur['LONGITUDE'],df_chandrapur['LATITUDE'])]
gdf=GeoDataFrame(chandrapur_div_df_sorted,geometry=geometry)
gdf2=GeoDataFrame(df_ballarshah,geometry=geometry2)
gdf3=GeoDataFrame(df_chandrapur,geometry=geometry3)

In [98]:
m_1=folium.Map(location=[20.2095, 79.5603],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()


        

for idx, row in gdf.iterrows():
    #folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']]).add_to(m_1)
    
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']]))

m_1.add_child(mc)
m_1.save('ruralDC.html')
m_1


In [99]:
m_2=folium.Map(location=[20.2095, 79.5603],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()

for idx, row in gdf.iterrows():
    consumer_name = str(row['CONSUMER NAME']) if pd.notnull(row['CONSUMER NAME']) else ''
    address = str(row['ADDRESS']) if pd.notnull(row['ADDRESS']) else ''
    mobile = str(row['MOBILE']) if pd.notnull(row['MOBILE']) else ''
    
    popup_text = consumer_name 
    popup1=folium.Popup(popup_text,parse_html=True)
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']],popup=popup1))

m_2.add_child(mc)
m_2.save('ballarshah_div_survey_withdetails.html')
m_2

In [103]:
m_4=folium.Map(location=[19.9792146, 79.3031043],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()

for idx, row in gdf.iterrows():
    consumer_name = str(row['CONSUMER NAME']) if pd.notnull(row['CONSUMER NAME']) else ''
    mobile = str(row['MOBILE']) if pd.notnull(row['MOBILE']) else ''
    google_map_link = '<a href="' + str(row['Googlemap']) + '">Google Map</a>'
    popup_text = consumer_name + ',' + mobile + ',' + google_map_link
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']], popup=popup_text))



m_4.add_child(mc)
m_4.save('ruralDC.html')
m_4




In [101]:
m_3=folium.Map(location=[19.9792146, 79.3031043],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()

for idx, row in gdf2.iterrows():
    #folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']]).add_to(m_1)
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']],popup=str(row['CONSUMER NAME'])+ ','+ str(row['MOBILE'])+',' + str(row['SECTION_NAME'])))


m_3.add_child(mc)
m_3.save('ballarshah_div_survey_withdetails.html')
m_3


In [102]:
gdf[gdf['SUB_DIVISION']=='CHANDRAPUR S/DN.'].sort_values('LATITUDE')

KeyError: 'SUB_DIVISION'

In [195]:
chandrapur_div_df_sorted['SUB_DIVISION'].unique()

array(['CHANDRAPUR-II(U)SDN', 'CHANDRAPUR S/DN.', 'MUL S/DN.',
       'SAOLI S/DN'], dtype=object)

In [25]:
#gdf_temp.drop(index=99,inplace=True)
gdf_temp.sort_values('LATITUDE')

,DIVISION,SUB_DIVISION,SECTION_NAME,CONSUMER NAME,ADDRESS,MOBILE,METER NO,LATITUDE,LONGITUDE,geometry
51,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,VITHHAL ZOLBAJI BHOYAR,SR. NO. 717 VG PIPARI DIST CHANDRAPUR,9.850402e+09,3732968.0,19.904987,79.204106,POINT (79.20411 19.90499)
71,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,RAJKUMAR VITHOBA NAGPURE,"SR.NO. 234 VG WENDALI TAL, DIST CHANDRAPUR",8.329062e+09,10018976.0,19.905074,79.204760,POINT (79.20476 19.90507)
53,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,REKHA SHANKAR KAMBALE,SR.NO 286 VG- SIDUR DIST CHANDRAPUR,9.767151e+09,3732779.0,19.905312,79.203886,POINT (79.20389 19.90531)
62,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,MURLIDHAR JAIRAM NANDEKAR,"SR.NO. 378 VG SIDUR TAL, DIST CHANDRAPUR",9.604155e+09,10018972.0,19.905391,79.205880,POINT (79.20588 19.90539)
59,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Ghugus DC,EKNATH NATTHU NIKHADE,SR.NO. 14 VG SIDUR GHUGHUS DC DIST CHANDRAPUR,7.875108e+09,3732772.0,19.905801,79.202065,POINT (79.20206 19.90580)
...,...,...,...,...,...,...,...,...,...,...
17,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,VANWASU RUSHI JUNGHARE,108/2 CHICHOLI CHANDRAPUR,9.405155e+09,10018610.0,20.519490,79.162870,POINT (79.16287 20.51949)
12,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,SHRI HANUMAN RUSHI ASUTKAR,121/2 CHICHOLI CHICHOLI CHANDRAPUR,9.623861e+09,10018384.0,20.525910,79.163323,POINT (79.16332 20.52591)
18,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,MANGESH NILKANTH ASUTKAR,121/3 CHICHOLI CHANDRAPUR,1.000000e+10,10018443.0,20.525910,79.163323,POINT (79.16332 20.52591)
15,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,PANDURANG NAMDEV UGHE,112/4 CHICHOLI CHANDRAPUR,1.000000e+10,10018386.0,20.615900,79.161215,POINT (79.16121 20.61590)


In [34]:

from folium.plugins import AntPath
gdf_temp=gdf[gdf['SUB_DIVISION']=='CHANDRAPUR S/DN.'].sort_values('LATITUDE')
gdf_temp.drop(index=99,inplace=True)
pathLatLngs=[(x,y) for x,y in zip(gdf_temp['LATITUDE'],gdf_temp['LONGITUDE'])]

m_3=folium.Map(location=[20.2095, 79.5603],tiles='openstreetmap',zoom_start=10)
mc=MarkerCluster()



# folium.Circle(radius=30000,
#               location=[19.9641678, 79.2752267],
#               color='yellow',
#               weight=5).add_to(m_3)

AntPath(pathLatLngs,delay=4000,weight=3,color='red').add_to(m_3)       

for idx, row in gdf_temp.iterrows():
    #folium.Marker(location=[row['LATITUDE'],row['LONGITUDE']]).add_to(m_1)
    
    mc.add_child(Marker([row['LATITUDE'], row['LONGITUDE']],popup=row['CONSUMER NAME']+ ',' +row['ADDRESS'] +','+ str(row['MOBILE'])))

m_3.add_child(mc)
#m_1.save('chanda_divison.html')
#m_3.save('SAOLI SDN.html')
m_3


KeyError: '[99] not found in axis'

In [198]:
gdf

,DIVISION,SUB_DIVISION,SECTION_NAME,CONSUMER NAME,ADDRESS,MOBILE,METER NO,LATITUDE,LONGITUDE,geometry
0,CHANDRAPUR DIVSION,CHANDRAPUR-II(U)SDN,MIDC DC,KAWADU LATARI ASUTKAR,SR.NO. 76/1 VG PADOLI OLD TAL DIST CHANDRAP...,8.411931e+09,3732639.0,19.964466,79.259874,POINT (79.25987 19.96447)
1,CHANDRAPUR DIVSION,CHANDRAPUR-II(U)SDN,MIDC DC,SHOBHA Shaliram Sadafale,s. no. 163 datala datala,9.850316e+09,3732637.0,19.963583,79.261096,POINT (79.26110 19.96358)
2,CHANDRAPUR DIVSION,CHANDRAPUR-II(U)SDN,MIDC DC,MEENABAI NANAJI ITANKAR,SR.NO.979/2 INFRONT OF PODDAR HIGH SCHOOL VG...,7.410577e+09,3732638.0,19.962301,79.256849,POINT (79.25685 19.96230)
3,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,SHANKAR PAIKAJEE GURNULE,213 CHORGAON CHANDRAPUR,9.767173e+09,10018398.0,20.015932,79.255771,POINT (79.25577 20.01593)
4,CHANDRAPUR DIVSION,CHANDRAPUR S/DN.,Rural DC,BANDU RAMCHANDRA AWARI,SRY NO 04 AMBHORA CHANDRAPUR,9.011289e+09,10018798.0,20.020494,79.266848,POINT (79.26685 20.02049)
...,...,...,...,...,...,...,...,...,...,...
379,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,THIVRU SUKURAM BHURSE,94 KERODA KERODA SAOLI,8.806598e+09,3732031.0,20.070448,79.887910,POINT (79.88791 20.07045)
380,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,LIYAKAT SIRAJ SHAIKH,700 MOKHALA MOKHALA SAOLI,9.595285e+09,3732989.0,20.118293,79.899315,POINT (79.89932 20.11829)
381,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,SHRIKRUSHNA LAXMAN BHURSE,646 HARAMBA HARAMBA SAOLI,9.158778e+09,3732035.0,19.990635,79.919555,POINT (79.91956 19.99064)
382,CHANDRAPUR DIVSION,SAOLI S/DN,Vyahad D/C,NAMDEV BABURAO WASEKAR,526 SAMDA BUJ SAMDA BUJ SAOLI,8.975600e+09,3732983.0,20.121165,79.949378,POINT (79.94938 20.12117)
